<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/screen_align.png" style="margin: 0 auto;">


<h1 style="text-align: center; font-size=58px;">Read Concerns</h1>

In this lesson we're going to discuss read concerns.

#### Read Concerns

- Represent different levels of "read isolation"
- Can be used to specify a consistent view of the database

Read concerns are similar to write concerns in that they both involve how many nodes have applied a database operation.

While write concerns affected the acknowledgement received by the driver after a write operation, read concerns affect the data returned by a read operation. Different read concerns are referred to as different levels of "read isolation," because you can essentially "isolate" a read from the rest of the database if the data being read has only been written to one of the nodes. If data can be read by clients before that data has been replicated to a majority of nodes, it's considered a low level of read isolation.

The read concern you choose will depend on how consistent your view of the database needs to be. If you can afford to read slightly stale data, then maybe a low level of read isolation might suit your application.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_read_concern_local.png" style="margin: 0 auto;">


By default, when an application sends a read to a replica set, Mongo uses `readConcern("local")` (point). From the perspective of the database client (point), the data read using `readConcern("local")` has only been written to this one (point) node. In the vast majority of cases, the data will **also** be written to the other nodes (point) in the set, but the client only has proof that this (point) one node applied the write.

This means that there's a chance, however slim, that the data (point) returned from this read will be rolled back. This would happen if, sometime after this (point) data is returned and before the secondaries have replicated that data, the primary goes down and a secondary gets elected to become the new primary. That would mean that one of these (point) two nodes, who haven't replicated the data yet, will be the new primary, and the old primary will be rolled back to match the state of the new primary.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_read_concern_majority.png" style="margin: 0 auto;">


So the default readConcern in MongoDB is `"local"`, which reads whatever copy of the data exists on this (point) node, regardless of whether or not the other nodes had replicated that data. And for the vast majority of reads, `readConcern("local")` will work just fine.

But we might want a higher level of consistency on some of our reads, which we can achieve with the readConcern called `"majority"`. When the database client sends a read to Mongo with `readConcern("majority")`, it can verifiably claim that the data it gets back (point) has been replicated to a majority of nodes the replica set. The benefit of this readConcern level is that once data has been replicated to a majority of nodes, it's super durable in the event of a failure. Even if the current primary (point) fails, this (point) secondary can be elected primary and then the data won't get rolled back.

One thing to note here: if the secondaries aren't done replicating data at the time that the primary receives this (point) write, then whatever copy of the data **has** been replicated to a majority of nodes in the set will be the data returned to the client.

This means that if my age (point) on the primary (point) is 66, but both of the secondaries still think that my age is 65, then the age 65 will be returned to the client. That's because in a replica set with three members, two nodes are required to constitute a majority.

So clearly, `readConcern("majority")` might return slightly stale data, but it provides a higher level of read isolation, so you can be more confident that the data you're reading won't get rolled back. For this reason it's most useful when reading mission-critical data, because lower levels of read isolation have a slightly higher chance of being rolled back in the event of an emergency. If your application's core functionality depends on one read, like checking a user's current account balance, then you probably want that read to have a higher durability.

## Summary

- The default read concern in MongoDB is `"local"`
    - This does not check that data has been replicated
- The read concern `"majority"` allows for more durable reads
    - This only returns data that has been replicated to a majority of nodes